In [ ]:
# 📅 İleri Makine Öğrenmesi (Safe Mode)
# 🎯 Hedef: Şehir bölgelerinin trafik davranışını tahmin etmek (XGBoost)
# 🛡️ Yöntem: Ham veri yerine optimize edilmiş 'dashboard_charts_hourly' tablosunu kullanır.
# ------------------------------------------------------------------------------

print("🚀 GÜVENLİ MODELLEME SÜRECİ BAŞLIYOR...")
print("="*60)

# 1. KURULUM VE AUTH
# ------------------------------------------------------------------------------
!pip install google-cloud-bigquery pandas numpy matplotlib seaborn scikit-learn xgboost -q

from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

auth.authenticate_user()
print("✅ Kimlik doğrulama tamamlandı.")

PROJECT_ID = "YOUR_PROJECT_ID"
DATASET_ID = "milano_mobile_2013"
client = bigquery.Client(project=PROJECT_ID)

# 2. VERİ ÇEKME (GÜVENLİ TABLO)
# ------------------------------------------------------------------------------
def fetch_safe_data():
    print("\n📊 VERİ ÇEKİLİYOR (dashboard_charts_hourly)...")

    # Sadece 0.03 MB veri okuyacak. Tamamen güvenli.
    query = f"""
    SELECT
        datetime,
        land_use_label as Cluster_Label, -- İş Merkezi, Konut vb.
        total_activity
    FROM `{PROJECT_ID}.{DATASET_ID}.dashboard_charts_hourly`
    WHERE land_use_label != 'Diğer' -- Gürültüyü temizle
    ORDER BY datetime
    """

    df = client.query(query).to_dataframe()

    # Feature Engineering (Python tarafında yapıyoruz, bedava işlem)
    df['hour'] = df['datetime'].dt.hour
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

    # Lag Features (Geçmiş veriyi ekleme)
    # Her bölge tipi için ayrı ayrı kaydırma yapmalıyız
    df['prev_hour'] = df.groupby('Cluster_Label')['total_activity'].shift(1)
    df['prev_24h'] = df.groupby('Cluster_Label')['total_activity'].shift(24)

    df = df.dropna() # İlk günün verisi (geçmişi olmadığı için) silinir

    print(f"✅ Veri Hazır: {len(df)} satır. (Maliyet: ~$0.00)")
    return df

# 3. MODEL EĞİTİMİ (XGBOOST)
# ------------------------------------------------------------------------------
def train_models(df):
    print("\n🤖 MODELLER EĞİTİLİYOR...")

    results_list = []
    trained_models = {}

    clusters = df['Cluster_Label'].unique()

    # Grafik alanı hazırla
    fig, axes = plt.subplots(len(clusters), 1, figsize=(12, 4 * len(clusters)))
    if len(clusters) == 1: axes = [axes]

    for i, cluster in enumerate(clusters):
        print(f"   📍 Bölge: {cluster} modelleniyor...")

        # O kümeye ait veriyi al
        data = df[df['Cluster_Label'] == cluster].copy()

        # Özellikler ve Hedef
        X = data[['hour', 'day_of_week', 'is_weekend', 'prev_hour', 'prev_24h']]
        y = data['total_activity']

        # Train/Test Split (Son %20'yi test için ayır - Zaman sırasına göre)
        split_idx = int(len(data) * 0.8)
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

        # Model
        model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=42)
        model.fit(X_train, y_train)

        # Tahmin
        y_pred = model.predict(X_test)

        # Başarı Metrikleri
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        results_list.append({
            'Bölge': cluster,
            'R2 Başarı Skoru': f"{r2:.4f}",
            'Ortalama Hata': f"{mae:.2f}"
        })

        # Görselleştirme
        ax = axes[i]
        ax.plot(y_test.values, label='Gerçek', color='blue', alpha=0.5)
        ax.plot(y_pred, label='YZ Tahmini', color='red', linestyle='--', linewidth=2)
        ax.set_title(f"{cluster} - Tahmin Performansı (R2: {r2:.2f})")
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Tahminleri sakla (BigQuery'e atmak için)
        data.loc[X_test.index, 'prediction'] = y_pred
        trained_models[cluster] = data.loc[X_test.index]

    plt.tight_layout()
    plt.show()

    print("\n🏆 PERFORMANS SONUÇLARI:")
    print(pd.DataFrame(results_list))

    return trained_models

# 4. SONUÇLARI BIGQUERY'E KAYDETME
# ------------------------------------------------------------------------------
def save_results(models_dict):
    print("\n💾 SONUÇLAR KAYDEDİLİYOR...")

    # Tüm tahminleri birleştir
    all_predictions = pd.concat(models_dict.values())

    # Sadece gerekli sütunlar
    final_df = all_predictions[['datetime', 'Cluster_Label', 'total_activity', 'prediction']]
    final_df.columns = ['datetime', 'zone_type', 'actual_value', 'predicted_value']

    # BigQuery'e yaz
    table_id = f"{PROJECT_ID}.{DATASET_ID}.analysis_ml_predictions_v2"
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

    job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
    job.result()

    print(f"✅ Tahmin tablosu oluşturuldu: {table_id}")
    print("👉 Looker Studio'da 'Gerçek vs Tahmin' grafiği için bu tabloyu kullanabilirsiniz.")

# --- ANA AKIŞ ---
df_safe = fetch_safe_data()
if df_safe is not None:
    prediction_results = train_models(df_safe)
    save_results(prediction_results)

In [ ]:
def create_fixed_connection_map_view():
    print("\n🗺️ REPORTING: BAĞLANTI HARİTASI (V2 - POINT-TO-POINT) OLUŞTURULUYOR...")
    print("="*60)

    # Connection Map için Point 1 ve Point 2'yi "Lat,Long" string formatında hazırlıyoruz.
    # Looker Studio bu formatı "Konum" olarak çok sever.

    query = f"""
    CREATE OR REPLACE VIEW `{PROJECT_ID}.{DATASET_ID}.reporting_connection_map_v2` AS
    WITH milano_center AS (
      SELECT
        ST_CENTROID(ST_GEOGFROMTEXT(geometry)) as point
      FROM `{PROJECT_ID}.{DATASET_ID}.province_boundaries`
      WHERE UPPER(province_name) = 'MILANO'
    ),
    province_centers AS (
      SELECT
        UPPER(TRIM(province_name)) as province_name,
        ST_CENTROID(ST_GEOGFROMTEXT(geometry)) as point
      FROM `{PROJECT_ID}.{DATASET_ID}.province_boundaries`
    )
    SELECT
      m.provinceName as target_province,

      -- Point 1: Origin (Milano) - GEOGRAPHY Tipi
      (SELECT point FROM milano_center) as origin_geopoint,
      -- Point 1: Origin (Milano) - STRING "Lat,Long" Tipi (Yedek Garantili)
      CONCAT(CAST(ST_Y((SELECT point FROM milano_center)) AS STRING), ',', CAST(ST_X((SELECT point FROM milano_center)) AS STRING)) as origin_latlong,

      -- Point 2: Destination (Hedef) - GEOGRAPHY Tipi
      p.point as dest_geopoint,
      -- Point 2: Destination (Hedef) - STRING "Lat,Long" Tipi (Yedek Garantili)
      CONCAT(CAST(ST_Y(p.point) AS STRING), ',', CAST(ST_X(p.point) AS STRING)) as dest_latlong,

      m.total_from_milano as flow_strength

    FROM `{PROJECT_ID}.{DATASET_ID}.province_movement_nature` m
    JOIN province_centers p
      ON UPPER(TRIM(m.provinceName)) = p.province_name
    WHERE m.total_from_milano > 0
      AND UPPER(m.provinceName) != 'MILANO'
    """

    try:
        client.query(query).result()
        print(f"✅ View oluşturuldu: {PROJECT_ID}.{DATASET_ID}.reporting_connection_map_v2")
        print("👉 Looker Studio'da 'reporting_connection_map_v2' tablosunu ekleyin.")
    except Exception as e:
        print(f"❌ Hata: {e}")

create_fixed_connection_map_view()